In [1]:
import pandas as pd
import numpy as np
import plotnine as p9
import matplotlib.pyplot as plt
import os


date_time = "2023-07-30_19:34"
date_time = "2023-07-31_19:32"


root_results_dir = "/projects/genomic-ml/da2343/ml_project_2/hpc/accuracy_tune/orders"
df = pd.read_csv(f"{root_results_dir}/{date_time}_results.csv")

In [2]:
folders_path = "/projects/genomic-ml/da2343/ml_project_2/data"
dataset_list = ["EURUSD_H1", "USDJPY_H1", "GBPUSD_H1",
                "AUDUSD_H1", "USDCAD_H1", "USDCHF_H1", 
                "EURJPY_H1", "EURGBP_H1"]


for dataset_name in dataset_list:
    raw_df = df[df['dataset_name'] == dataset_name]
    # remove the _H1 from the dataset_name
    symbol = dataset_name[:-3]
    
    raw_df_dir = os.path.join(folders_path, symbol, f"{symbol}_H1_2011_2015_TRADES.csv")
    raw_df.to_csv(raw_df_dir, index=False)
    
    wrangled_df_dir = os.path.join(folders_path, symbol, f"{symbol}_H1_2011_2015_TRADES_REAL.csv")
    # wrangled_df = raw_df[['position', 'RSI', 'ADX', 'WILLR', 'AROON_Oscillator', 'label']]
    wrangled_df = raw_df[['position', 'RSI', 'ADX','label']]
    wrangled_df.reset_index(drop=True, inplace=True)
    wrangled_df = wrangled_df.assign(index=wrangled_df.index)
    wrangled_df.to_csv(wrangled_df_dir, index=False)
    
# print(wrangled_df)
wrangled_df


,position,RSI,ADX,label,index
0,0.0,46.441976,19.158403,1.0,0
1,0.0,50.657939,19.397116,1.0,1
2,0.0,49.783929,18.233699,1.0,2
3,0.0,60.655018,43.985430,1.0,3
4,1.0,55.387421,33.485337,0.0,4
...,...,...,...,...,...
396,1.0,43.688536,43.200046,0.0,396
397,0.0,51.654539,20.550078,1.0,397
398,0.0,53.487342,27.270764,0.0,398
399,0.0,48.171362,18.286948,0.0,399


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, PowerTransformer

trade_df_test = pd.read_csv('/projects/genomic-ml/da2343/ml_project_2/data/EURUSD/EURUSD_H1_2016_2022_TRADES.csv')
# trade_df_test = pd.read_csv('/projects/genomic-ml/da2343/ml_project_2/hpc/accuracy_tune/orders/2023-07-30_19:34_results.csv')
trade_df_test = trade_df_test[['position', 'RSI', 'ADX', 'label', 'year']]
trade_df_test = trade_df_test[trade_df_test['year'] == 2021]
trade_df_test.drop('year', axis=1, inplace=True)


# trade_df = pd.read_csv('/projects/genomic-ml/da2343/ml_project_2/data/EURUSD/EURUSD_H1_2011_2015_TRADES.csv')
# trade_df = trade_df[[ 'MACD', 'MACD_Signal', 'MACD_Hist', 'RSI', 'ADX', 'AROON_Oscillator', 'WILLR', 'label']]
# trade_df = trade_df[['position', 'MACD', 'MACD_Signal', 'MACD_Hist', 'RSI', 'ADX', 'AROON_Oscillator', 'WILLR', 'label']]
# trade_df = trade_df[['position', 'RSI', 'ADX', 'WILLR', 'label']]
# trade_df_test = trade_df_test[['position', 'RSI', 'ADX', 'label']]


# reset index
# trade_df.reset_index(drop=True, inplace=True)

# add an index column to trade_df
# trade_df['index'] = trade_df.index

# save trade_df to csv
# trade_df.to_csv('/projects/genomic-ml/da2343/ml_project_2/data/EURUSD/EURUSD_H1_2011_2015_TRADES_binary.csv', index=False)

In [ ]:
trade_df_test

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, PowerTransformer

trade_df = pd.read_csv('/projects/genomic-ml/da2343/ml_project_2/data/EURUSD/EURUSD_H1_2011_2015_TRADES_synthetic.csv')
# drop index column
# trade_df = trade_df.drop(columns=['index'])

In [ ]:
trade_df

In [ ]:

import pandas as pd
import numpy as np
import plotnine as p9
from sklearn.model_selection import *
from sklearn.feature_selection import *
from sklearn.preprocessing import *
from sklearn.pipeline import *

from sklearn.linear_model import *
from sklearn.neighbors import *
from sklearn.model_selection import *
from sklearn.dummy import DummyClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import *
import matplotlib.pyplot as plt
import matplotlib
import plotnine as p9

np.set_printoptions(threshold=np.inf)


X_train, y_train = trade_df.iloc[:, :-1].to_numpy(), trade_df.iloc[:, -1]
X_test, y_test = trade_df_test.iloc[:, :-1].to_numpy(), trade_df_test.iloc[:, -1]

model = LogisticRegressionCV()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
tn_accuracy = tn / (tn + fp)
            
test_acc_dict = {
    "test_accuracy_percent": accuracy_score(y_test, y_pred) * 100,
    "tn_accuracy": tn_accuracy * 100,
    # "precision_score": precision_score(y_test, y_pred),
    # "f1_score": f1_score(y_test, y_pred),
    # "algorithm": algorithm
}

print(test_acc_dict)

In [ ]:

# FORECASTING + MACD CROSSOVER
# {'test_accuracy_percent': 55.66218809980806, 'tn_accuracy': 69.64285714285714} for all the years

# {'test_accuracy_percent': 52.87356321839081, 'tn_accuracy': 71.15384615384616} for 2022
# {'test_accuracy_percent': 39.34426229508197, 'tn_accuracy': 65.21739130434783} for 2021
# {'test_accuracy_percent': 58.46153846153847, 'tn_accuracy': 76.19047619047619} for 2020
# {'test_accuracy_percent': 56.25, 'tn_accuracy': 58.92857142857143} for 2019
# {'test_accuracy_percent': 60.49382716049383, 'tn_accuracy': 70.49180327868852} for 2018
# {'test_accuracy_percent': 60.526315789473685, 'tn_accuracy': 76.19047619047619} for 2017
# {'test_accuracy_percent': 58.620689655172406, 'tn_accuracy': 70.0} for 2016

# MACD CROSSOVER
# {'test_accuracy_percent': 55.24475524475524, 'tn_accuracy': 69.58904109589041} just MACD crossover

# {'test_accuracy_percent': 51.06382978723404, 'tn_accuracy': 71.69811320754717} for 2022
# {'test_accuracy_percent': 41.17647058823529, 'tn_accuracy': 62.06896551724138} for 2021
# {'test_accuracy_percent': 54.929577464788736, 'tn_accuracy': 75.0} for 2020
# {'test_accuracy_percent': 56.71641791044776, 'tn_accuracy': 59.32203389830508} for 2019
# {'test_accuracy_percent': 58.139534883720934, 'tn_accuracy': 67.6923076923077} for 2018
# {'test_accuracy_percent': 61.05263157894737, 'tn_accuracy': 79.24528301886792} for 2017
# {'test_accuracy_percent': 60.43956043956044, 'tn_accuracy': 70.96774193548387} for 2016


In [ ]:
arr = y_pred
num_of_ones = 0
for element in arr:
    if element == 1:
        num_of_ones += 1
print(num_of_ones)


In [ ]:
test_acc_dict

# Using KFOLD to train and test the various model

In [ ]:
data_dict = {
    'EURUSD_TRADES': (trade_df.iloc[:, :-1].to_numpy(), trade_df.iloc[:, -1] ),
    # 'EURUSD_TRADES_SS': (StandardScaler().fit_transform(trade_df.iloc[:, :-1].to_numpy()), trade_df.iloc[:, -1] ),
    # 'EURUSD_TRADES_PT': (PowerTransformer().fit_transform(trade_df.iloc[:, :-1].to_numpy()), trade_df.iloc[:, -1] ),
}

In [ ]:
import pandas as pd
import numpy as np
import plotnine as p9
from sklearn.model_selection import *
from sklearn.feature_selection import *
from sklearn.preprocessing import *
from sklearn.pipeline import *

from sklearn.linear_model import *
from sklearn.neighbors import *
from sklearn.model_selection import *
from sklearn.dummy import DummyClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import *
import matplotlib.pyplot as plt
import matplotlib
import plotnine as p9

np.set_printoptions(threshold=np.inf)

test_acc_df_list = []


classifier_dict = {
    "KNeighborsClassifier": KNeighborsClassifier(10),
    # "SVC": SVC(kernel="linear", C=0.025),
    # "SVC": SVC(gamma=2, C=1),
    # "GaussianProcessClassifier": GaussianProcessClassifier(1.0 * RBF(1.0)),
    # "DecisionTreeClassifier": DecisionTreeClassifier(max_depth=5),
    # "MLPClassifier": MLPClassifier(),
    # "AdaBoostClassifier": AdaBoostClassifier(),
    # "GaussianNB": GaussianNB(),
    # "QuadraticDiscriminantAnalysis": QuadraticDiscriminantAnalysis(),
    "RandomForestClassifier": RandomForestClassifier(),
    'DummyClassifier': DummyClassifier(strategy="uniform"),
    'LogisticRegressionCV': LogisticRegressionCV(cv=3, random_state=0, max_iter=1_000),
    
}

for data_set, (input_mat, output_vec) in data_dict.items():
    kf = KFold(n_splits=3, shuffle=True, random_state=1)
    for fold_id, (train_index, test_index) in enumerate(kf.split(input_mat)):
        X_train, X_test = input_mat[train_index], input_mat[test_index]
        y_train, y_test = output_vec[train_index], output_vec[test_index]

        pred_dict = {}
        # iterate over classifiers
        for name, clf in classifier_dict.items():
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            pred_dict[name] = y_pred
        
        for algorithm, y_pred in pred_dict.items():
            tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
            tn_accuracy = tn / (tn + fp)
            
            test_acc_dict = {
                "test_accuracy_percent": accuracy_score(y_test, y_pred) * 100,
                "tn_accuracy": tn_accuracy * 100,
                # "precision_score": precision_score(y_test, y_pred),
                # "f1_score": f1_score(y_test, y_pred),
                "data_set": data_set,
                "fold_id": fold_id,
                "algorithm": algorithm
            }
            test_acc_df_list.append(pd.DataFrame(test_acc_dict, index=[0]))
test_acc_df = pd.concat(test_acc_df_list)


test_acc_df = pd.concat(test_acc_df_list)
gg = p9.ggplot() +\
    p9.geom_point(
        p9.aes(
            x="tn_accuracy",
            y="algorithm",
            color = "factor(fold_id)",
        ),
        data=test_acc_df) +\
    p9.facet_wrap("data_set") +\
    p9.scale_color_manual(values=["red", "blue", "green", "yellow", "pink"], name= "Fold ID")
gg


In [ ]:
y_pred

In [ ]:
mean_test_acc_df = test_acc_df.groupby(["data_set", "algorithm"]).mean().reset_index()


gg = p9.ggplot() +\
    p9.geom_point(
        p9.aes(
            x="tn_accuracy",
            y="algorithm",
            color = "factor(fold_id)",
        ),
        data=mean_test_acc_df) +\
    p9.facet_wrap("data_set") +\
    p9.scale_color_manual(values=["red", "blue", "green"], name= "Fold ID")
gg
